In [4]:
import osmnx as ox
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import random

# ------------------------------
# 1. Download road network (Milano centro)
# ------------------------------
G = ox.graph_from_place("Milan, Italy", network_type="drive")
G = nx.convert_node_labels_to_integers(G)  # rinumera i nodi per semplicità

nodes = list(G.nodes)
id2coord = {i: (G.nodes[i]['y'], G.nodes[i]['x']) for i in nodes}  # {id: (lat, lon)}
max_node = max(nodes)

# ------------------------------
# 2. Create training sequences (random walks on road network)
# ------------------------------
def random_walk(G, start, length=15):
    walk = [start]
    for _ in range(length-1):
        neighbors = list(G.neighbors(walk[-1]))
        if not neighbors:
            break
        walk.append(random.choice(neighbors))
    return walk

# generate random walks
raw_trajectories = [random_walk(G, random.choice(nodes), length=15) for _ in range(200)]

# ------------------------------
# 2b. Pad trajectories to fixed length
# ------------------------------
seq_len = 15
pad_token = max_node + 1  # token di padding

padded_trajectories = []
for traj in raw_trajectories:
    if len(traj) < seq_len:
        traj = traj + [pad_token]*(seq_len - len(traj))
    else:
        traj = traj[:seq_len]
    padded_trajectories.append(traj)

# Convert to tensor
data = torch.tensor(padded_trajectories, dtype=torch.long)

# ------------------------------
# 3. Transformer model (predict next road node)
# ------------------------------
class RoadTransformer(nn.Module):
    def __init__(self, num_nodes, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embed = nn.Embedding(num_nodes+2, d_model)  # +1 per padding
        layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.encoder = nn.TransformerEncoder(layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_nodes+2)

    def forward(self, x):
        x = self.embed(x)          # (batch, seq, d_model)
        x = x.permute(1,0,2)       # (seq, batch, d_model)
        x = self.encoder(x)
        x = x.permute(1,0,2)       # (batch, seq, d_model)
        return self.fc(x)

model = RoadTransformer(num_nodes=max_node)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_token)  # ignora il padding

# ------------------------------
# 4. Training
# ------------------------------
for epoch in range(50):
    optimizer.zero_grad()
    inp, target = data[:, :-1], data[:, 1:]
    out = model(inp)  # (batch, seq-1, num_nodes+1)
    loss = loss_fn(out.reshape(-1, max_node+2), target.reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# ------------------------------
# 5. Generate synthetic trajectory
# ------------------------------
model.eval()
with torch.no_grad():
    start_node = random.choice(nodes)
    generated = [start_node]
    cur = torch.tensor([[start_node]], dtype=torch.long)

    for _ in range(10):  # genera 10 step
        out = model(cur)[:, -1, :]          # ultimo step
        next_node = torch.argmax(out, dim=-1).item()
        if next_node == pad_token:  # stop se padding
            break
        generated.append(next_node)
        cur = torch.cat([cur, torch.tensor([[next_node]])], dim=1)

    # Convert back to GPS coords
    synthetic_coords = [id2coord[n] for n in generated]

print("Synthetic trajectory (lat, lon):")
for c in synthetic_coords:
    print(c)

# ------------------------------
# 6. Optional: visualize with Folium
# ------------------------------
import folium

def plot_on_map(coords, city_center=(45.4642, 9.19), zoom_start=14):
    m = folium.Map(location=city_center, zoom_start=zoom_start)
    folium.PolyLine(coords, color="blue", weight=4, opacity=0.7).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color="red")).add_to(m)
    return m

map_obj = plot_on_map(synthetic_coords)
map_obj.save("synthetic_trajectory.html")
print("✅ Mappa salvata come synthetic_trajectory.html")

/Users/fbalicchia/miniconda3/envs/timeseries-model/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 0, Loss: 9.6169
Epoch 10, Loss: 8.2654
Epoch 20, Loss: 7.2883
Epoch 30, Loss: 6.3125
Epoch 40, Loss: 5.4367
Synthetic trajectory (lat, lon):
(45.4811452, 9.1556957)
(45.4788849, 9.1341494)
(45.4780434, 9.1352739)
(45.4772549, 9.1363071)
(45.4760601, 9.1371358)
(45.4760765, 9.1355121)
(45.4761038, 9.1337952)
(45.5041787, 9.1579496)
(45.5023165, 9.1582489)
(45.5041787, 9.1579496)
(45.5023165, 9.1582489)
✅ Mappa salvata come synthetic_trajectory.html
